In [1]:
with open("../data/day24.txt") as f:
    data = f.read()

sections = data.split("\n\n")

starting_values = [x.split(": ") for x in sections[0].split("\n")]
operations = [x.split(" -> ") for x in sections[1].split("\n")]
operations = {x[1]: x[0].split() for x in operations}

starting_values = {x[0]: int(x[1]) for x in starting_values}

In [2]:
def handle_operation(desired_value, operations, current_values):
    operation = operations[desired_value]
    name1, operator, name2 = operation
    try:
        value_1 = current_values[name1]
    except KeyError:
        value_1 = handle_operation(name1, operations, current_values)
    try:
        value_2 = current_values[operation[2]]
    except KeyError:
        value_2 = handle_operation(name2, operations, current_values)
    outc = execute_operation(operator, value_1, value_2)
    return outc


def execute_operation(operator, value_1, value_2):
    if operator == "AND":
        return value_1 & value_2
    if operator == "OR":
        return value_1 | value_2
    if operator == "XOR":
        return value_1 ^ value_2


def get_result(operations, starting_values):
    result = starting_values.copy()

    ans = 0
    for i in range(100):
        zval = "z" + f"{i}".zfill(2)
        try:
            operations[zval]
        except KeyError:
            return ans

        result[zval] = handle_operation(zval, operations, result)
        ans += result[zval] * (2**i)
    return ans


get_result(operations, starting_values)

49430469426918

# Part 2

In [3]:
## 0111 + 0111
# 1110

# 0101 + 0111
# 1100


# 0110 + 0110
# 1100

In [4]:
# z_i = (x_i XOR y_i) XOR (x_i-1 AND y_i-1)

In [5]:
# z = XOR_zelf XOR Overflow_-1
# v = XOR_zelf (# own value)
# o = overflow_zelf = AND_zelf  (# overflow own bit)
# n = XOR_zelf AND overflow - 1 = v AND O-1  (# overflow next bit)
# O = o OR n (# Any overflow)
# z = v XOR O-1
#
# overflow = AND_zelf OR (XOR_zelf AND overflow - 1)
#

# z = v XOR (o-1 OR (v-1 AND O-2))

# # overflow - 1 = AND-1 OR (XOR-1 AND overflow - 2)
# overflow[0] = 0

In [6]:
# z2 = v2 XOR O1
# z2 = v2 XOR (o1 OR n1) = v2 XOR (o1 OR (v1 AND o0))

In [7]:
# n1 = ('y00', 'AND', 'x00'), 'AND', ('y01', 'XOR', 'x01') = o-1 AND v1

In [8]:
def find_underlying_code(desired_value, operations):
    operation = operations[desired_value]
    name1, operator, name2 = operation
    if name1[0] not in ("x", "y"):
        name1 = find_underlying_code(name1, operations)
    if name2[0] not in ("x", "y"):
        name2 = find_underlying_code(name2, operations)
    return name1, operator, name2


def reduce(name1, operator, name2):
    # name1, operator, name2 = operation[0], operation[1], operation[2]
    if isinstance(name1, tuple):
        name1 = reduce(*name1)
    if isinstance(name2, tuple):
        name2 = reduce(*name2)

    values = sorted([name1, name2])
    types = (values[0][0], values[1][0])
    levels = [int(values[0][1:]), int(values[1][1:])]

    if types == ("x", "y"):
        if len(set(levels)) > 1:
            raise ValueError(f"Not matching levels for {types=}: {levels=}")
        level = str(levels[0]).zfill(2)
        if operator == "XOR":
            if int(level) == 0:
                return "z00"
            return "v" + level  # own value
        if operator == "AND":
            if int(level) == 0:
                return "O00"  # Big O means: overflow of all earlier bits
            return "o" + level  # overflow of bits directly next to it
    if types == ("O", "v"):
        if (levels[1] - levels[0]) != 1:
            raise ValueError(f"Not matching levels for {types=}: {levels=}")
        level = str(levels[1]).zfill(2)
        if operator == "XOR":
            return "z" + level
        if operator == "AND":
            return "n" + level

    if types == ("O", "o"):
        if (levels[1] - levels[0]) != 1:
            raise ValueError(f"Not matching levels for {types=}: {levels=}")
        if operator == "OR":
            level = str(levels[1]).zfill(2)
            return "O" + level

    if types == ("n", "o"):
        if levels[1] != levels[0]:
            raise ValueError(f"Not matching levels for {types=}: {levels=}")
        if operator == "OR":
            level = str(levels[1]).zfill(2)
            return "O" + level
    return f"({name1} {operator} {name2})"

    # str(level).zfill(2)
    # return reducing[(values, operator)]


value_1, operator, value_2 = find_underlying_code("z11", operations)
# print(value_1, operator, value_2)
reduce(value_1, operator, value_2)

# z = XOR_zelf XOR Overflow_-1
# v = XOR_zelf (# own value)
# o = overflow_zelf = AND_zelf  (# overflow own bit)
# n = XOR_zelf AND overflow - 1 = v AND O-1  (# overflow next bit)
# O = o OR n (# Any overflow)
# z = v XOR O-1
#

'(O10 XOR o11)'

In [9]:
def find_underlying_values(desired_value, operations, underlying_values=None):
    if underlying_values is None:
        underlying_values = []
    operation = operations[desired_value]
    name1, operator, name2 = operation

    if name1[0] not in ("x", "y"):
        underlying_values.append(name1)
        more_values = find_underlying_values(name1, operations)
        underlying_values.extend(more_values)
    if name2[0] not in ("x", "y"):
        underlying_values.append(name2)
        more_values = find_underlying_values(name2, operations)
        underlying_values.extend(more_values)
    return set(underlying_values)


find_underlying_values("z01", operations)

{'bwv', 'tcd'}

In [10]:
switched_operations = set()
correct_values = set()

new_operations = operations.copy()
for i in range(46):
    outc = "z" + str(i).zfill(2)
    value_1, operator, value_2 = find_underlying_code(outc, new_operations)
    underlying_values = find_underlying_values(outc, new_operations)
    underlying_values.add(outc)

    correct = reduce(value_1, operator, value_2) == outc
    print(i, correct)

    if not correct:
        possible_wrong = underlying_values.difference(correct_values)

        all_possible = set(new_operations.keys()).difference(correct_values)
        for try_ in possible_wrong:
            if correct:
                continue
            for other in all_possible:
                try_operations = new_operations.copy()

                try_operations[try_] = try_operations[other]
                try_operations[other] = try_operations[try_]
                try:
                    value_1, operator, value_2 = find_underlying_code(
                        outc, try_operations
                    )
                except (
                    RecursionError
                ):  # This is incredibly ugly, but surprisingly still fast enough
                    continue
                try:
                    reduced = reduce(value_1, operator, value_2)
                except ValueError:
                    continue
                correct = reduced == outc
                if correct:
                    switched_operations.add((try_, other))
                    new_operations[try_] = operations[other]
                    new_operations[other] = operations[try_]
                    print(switched_operations)
                    break
    for val in underlying_values:
        correct_values.add(val)


def print_final_result(switched_operations):
    result = []
    for combi in switched_operations:
        result.extend(combi)
    return ",".join(sorted(result))


print_final_result(switched_operations)

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 False
{('qnw', 'qff')}
12 True
13 True
14 True
15 True
16 False
{('z16', 'pbv'), ('qnw', 'qff')}
17 True
18 True
19 True
20 True
21 True
22 True
23 False
{('z16', 'pbv'), ('qnw', 'qff'), ('z23', 'qqp')}
24 True
25 True
26 True
27 True
28 True
29 True
30 True
31 True
32 True
33 True
34 True
35 True
36 False
{('z16', 'pbv'), ('qnw', 'qff'), ('z36', 'fbq'), ('z23', 'qqp')}
37 True
38 True
39 True
40 True
41 True
42 True
43 True
44 True
45 False


'fbq,pbv,qff,qnw,qqp,z16,z23,z36'